In [1]:
from arcgis import GIS
from arcgis.gis import Item
import pandas as pd
from time import sleep

In [2]:
gis = GIS(username="username")


Enter password: ········


In [ ]:
global data
data = []

In [21]:
def getServicesFromWebMap(itemId, appTitle,appID):
    try:
        item = gis.content.get(itemId)
        itemProp = item.get_data()
        for lyr in itemProp["operationalLayers"]:
            
            values = list(lyr.keys())
            if "url" in values and "itemId" in values:
                serviceI2 = lyr["itemId"]
            elif "url" in values :
                serviceI2 = lyr["url"]
            else:
                serviceI2 = ("Cargada directamente")
            data.append([appTitle, appID, item.title, item.itemid, lyr["title"], serviceI2])
            
    except: 
        print("Not have services")

In [22]:
def getMapsFromApps(item, appTitle, appID):
    try:
        keywords = item.typeKeywords
        if "Dashboard" in keywords:
            maps = [mapWidget for mapWidget in item.get_data()["widgets"] if mapWidget["type"] == "mapWidget"]
            for webMaps in maps:    
                itemId = webMaps["itemId"]
                getServicesFromWebMap(itemId, appTitle, appID)
        elif "Web AppBuilder" in keywords:
            itemId = item.get_data()["map"]["itemId"]
            getServicesFromWebMap(itemId, appTitle, appID)
        elif "Story Map" in keywords:
            for content in item.get_data()["values"]["story"]["entries"]:
                itemId = content["media"]["webmap"]["id"]
                getServicesFromWebMap(itemId, appTitle, appID)                    
        else:
            itemId = item.get_data()["values"]["webmap"]
            getServicesFromWebMap(itemId, appTitle, appID)
    except:
        print("App not have map")

In [29]:
def dfUserAppAndMaps(user):    
    foldersName = [None]+[folder["title"] for folder in user.folders]
    for folder in foldersName:
        items = user.items(folder)
        for item in items:
            
            if item.type in ["Dashboard", "Web Mapping Application"]:
                getMapsFromApps(item, item.title, item.itemid)
            sleep(1)
    columns = ["App Title", "App ID", "Map Title", "Map ID", "Service Title", "Service ID/ URL"]
    df = pd.DataFrame(data, columns=columns)
    return df
      

In [27]:
def main():
    with pd.ExcelWriter('Children Services.xlsx', engine='xlsxwriter') as writer:     
        for user in gis.users.search():
            df = dfUserAppAndMaps(user)
            df.to_excel(writer, sheet_name=user.fullName)
            sleep(3)
            del df

In [ ]:
main()